# Business Case Study: Home Depot, Lowes, Tool Time

## Background

##### In this case study, we will be put in the shoes of a consultant looking to understand some of the factors that Lowe's and Home Depot look to when deciding to build new stores. We will then use this strategy to advise a new competitor we will call 'Tool Time' on where this new entrant should build its next five stores based on this information on Home Depot and Lowe's.


## Case Study Outline 

1. **Exploratory Data Analysis & creating a map of stores**
1. **Identifying relationships between variables with linear regression model**
1. **Leveraging linear regression model to perform _new store location predictions_ for Lowe's, Home Depot and Tool Time**
1. **Comparing our results to realtor.com 'market hotness' data** 


## Questions:

### Initial data load

In [ ]:
import pandas as pd 
import numpy as np 

In [ ]:
#cd'ing to the data directory and viewing available data files 
%cd ./data/
%ls

In [ ]:
hdlo = pd.read_csv("Home_Depot_Lowes_Data.csv", sep = ',')

In [ ]:
pd.set_option('display.max_columns', 27)

### Question 1: Perform Exploratory Data Analysis on the stores

**a. What are the total store counts of Home Depot and Lowes?**

In [ ]:
print('There are a total of {} Lowe\'s stores in this dataset'.format(np.sum(hdlo.Lcount)))

In [ ]:
print('There are a total of {} HDSupply stores in this dataset'.format(np.sum(hdlo.HDcount)))

**b. Create one dummy variable for Home Depot and one dummy variable for Lowes that identifies if the store is located in a county**

In [ ]:
hdlo['HD_dummy'] = (hdlo['HDcount'] > 0) * 1

In [ ]:
hdlo['Lo_dummy'] = (hdlo['Lcount'] > 0) * 1

**c. Which store is present in more counties?**

In [ ]:
print('Lowe\'s stores are present in {} counties'.format(np.sum(hdlo.Lo_dummy)))
print('HDSupply stores are present in {} counties'.format(np.sum(hdlo.HD_dummy)))
print('HDSupply have a presence in a higher number of counties than do Lowe\'s stores')

### Question 2: Use a United States map with FIPS locations to plot the store locations of both Lowes and Home Depot

In [ ]:
import plotly.express as px

fig = px.choropleth(hdlo, geojson=counties, locations='county', color='HDcount',
                           color_continuous_scale="Viridis",
                           range_color=(0, 48),
                           scope="usa",
                           labels={'HDcount':'Home Depot stores'},
                           title = 'Home Depot Stores in the United States')
fig.update_layout(margin={"r":0,"t":50,"l":0,"b":0})
fig.show()

In [ ]:
fig = px.choropleth(hdlo, geojson=counties, locations='county', color='Lcount',
                           color_continuous_scale="Viridis",
                           range_color=(0, hdlo.Lcount.max()),
                           scope="usa",
                           labels={'Lcount':'Lowe\'s stores'},
                           title = 'Lowe\'s Stores in the United States')
fig.update_layout(margin={"r":0,"t":50,"l":0,"b":0})
fig.show()

**a. What observations can you make from the map?**

In [ ]:
#There tend to be high concentrations of Lowe's and Home Depot stores in metropolis and other high density areas. 

### Question 3: Create a linear regression model to identify the correlations among the variables.

In [ ]:
#imputing our data
hdlo['pct_U18_2000'].fillna(hdlo.pct_U18_2000.mean(), inplace = True)
hdlo['pct_U18_2010'].fillna(hdlo.pct_U18_2000.mean(), inplace = True)
hdlo['pctwhite_2000'].fillna(hdlo.pct_U18_2000.mean(), inplace = True)
hdlo['pctwhite_2010'].fillna(hdlo.pct_U18_2000.mean(), inplace = True)
hdlo['pctblack_2000'].fillna(hdlo.pct_U18_2000.mean(), inplace = True)
hdlo['pctblack_2010'].fillna(hdlo.pct_U18_2000.mean(), inplace = True)


In [ ]:
#further cleaning and separating our data 
hd_target = hdlo.HDcount
lo_target = hdlo.Lcount
hd_data = hdlo.copy()
lo_data = hdlo.copy()
hd_data.drop(['areaname','county','state','r1', 'r2', 'HD_dummy', 'Lo_dummy','HDcount','Lcount'], axis = 1, inplace = True)
lo_data.drop(['areaname','county','state','r1', 'r2', 'HD_dummy', 'Lo_dummy','HDcount','Lcount'], axis = 1, inplace = True)

In [ ]:
#fitting our linear regression model
from sklearn.linear_model import LinearRegression
lr1 = LinearRegression(normalize=True).fit(hd_data, hd_target)
print('Home Depot Regression Coefficients are:', lr1.coef_)
lr2 = LinearRegression(normalize=True).fit(lo_data, lo_target)
print('Lowe\'s Regression Coefficients are:', lr1.coef_)

In [ ]:
print('R^2 of Home Depot regression is',lr1.score(hd_data, hd_target))
print('R^2 of Lowes regression is',lr1.score(lo_data, lo_target))

In [ ]:
from sklearn.model_selection import cross_val_score

print(cross_val_score(lr1, hd_data, hd_target, cv = 5, n_jobs = -1))
print(cross_val_score(lr2, lo_data, lo_target, cv = 5, n_jobs = -1))

**a. What customer demographic variables are most import to Lowes?**

In [ ]:
hd_scores = list(zip(hd_data.columns, lr1.coef_))
lo_scores = list(zip(lo_data.columns, lr2.coef_))

In [ ]:
sorted(lo_scores, key = lambda x: x[1], reverse = True)

Lowe's is most interested in building new stores in areas where a large portion of the population is under the age of 18. This may be due to the fact that this may be an indicator that this county has high amounts of young families looking to potentially invest more down the road in home improvement. Lowe's is also interested in areas with high racial diversity and high rates of college education 

**b. What customer demographic variables are most import to Home Depot?**

In [ ]:
sorted(hd_scores, key = lambda x: x[1], reverse = True)

Home Depot is most interested in building new stores in areas with high degrees of racial diversity. Interestingly, Home Depot actually builds less stores all else equal in areas with large amounts of the population that is under 18 years old, in contract to Lowe's which chooses to build more houses in areas with more <18yo people as a percentage of the population all else equal 

**c. How are the chains similar in their decision making?**

As percent white and percent black variables appear in the top three variables of each regression analysis, it appears both chains place a heavy emphasis on areas with high degrees of racial diversity. Additionally, with pct_U18 and pctcollege appearing in the top five variables for each, counties with relatively higher amounts of young children, perhaps an indicator of families looking to further extend / renovate their homes, and relatively higher amounts of college educated people, perhaps an indicator of higher income households, also seem to be targets for new store construction for each chain. 

**d. How are they different?**

Home Depot values home owners percentage in 2000 higher than Lowes. In addition, Lowes seems to value 2000 density more than Home Depot. It's interesting that both chains values the 2000 demographic statistics over the 2010 data.

### Question 4:	What are the top 5 towns / cities that can be predicted as potential candidates for new locations for both Lowes and Home Depot? 

In [ ]:
#First way of answering this question - using the predict method of the LinearRegression class 
#to output how many stores in each area should be built, and returning the areas where 
#the most stores should be built according to our model 

nohd = hdlo.loc[hdlo['HDcount'] == 0]
nohd.drop(['areaname','county','state','r1', 'r2', 'HD_dummy', 'Lo_dummy','HDcount','Lcount'], axis = 1, inplace = True)
answers = zip(hdlo.loc[hdlo['HDcount'] == 0]['areaname'], list(lr1.predict(nohd)))
sorted(answers, key = lambda x: x[1], reverse = True)[:5]

In [ ]:
nolo = hdlo.loc[hdlo['Lcount'] == 0]
nolo.drop(['areaname','county','state','r1', 'r2', 'HD_dummy', 'Lo_dummy','HDcount','Lcount'], axis = 1, inplace = True)
answers = zip(hdlo.loc[hdlo['Lcount'] == 0]['areaname'], list(lr2.predict(nolo)))
sorted(answers, key = lambda x: x[1], reverse = True)[:5]

In [ ]:
hd_data.columns

In [ ]:
#Second way of answering this question - building Logistic Regression models that predict 1 = area where store 
#should be built and 0 = area where store should not be built (differs from Linear Regression prediction in that 
#you are instead predicting whether an area should have a store built, instead of how many stores should be built there)

In [ ]:
from sklearn.model_selection import train_test_split
#preparing our data 
hd_x = hdlo.copy()
hd_y = hdlo.HD_dummy
lo_x = hdlo.copy()
lo_y = hdlo.Lo_dummy
hd_x.drop(['areaname','county','state','r1', 'r2', 'HD_dummy', 'Lo_dummy','HDcount','Lcount'], axis = 1, inplace = True)
lo_x.drop(['areaname','county','state','r1', 'r2', 'HD_dummy', 'Lo_dummy','HDcount','Lcount'], axis = 1, inplace = True)

hdxtrain, hdxtest, hdytrain, hdytest = train_test_split(hd_x, hd_y, test_size = 0.3)
loxtrain, loxtest, loytrain, loytest = train_test_split(lo_x, lo_y, test_size = 0.33)

In [ ]:
#training our models 
from sklearn.linear_model import LogisticRegressionCV

lg1 = LogisticRegressionCV(Cs=10, cv=5, class_weight='balanced', max_iter=100, random_state=24)
lg2 = LogisticRegressionCV(Cs=10, cv=5, class_weight='balanced', max_iter=100, random_state=24)
lg1.fit(hdxtrain, hdytrain)
lg2.fit(loxtrain, loytrain)

In [ ]:
#predict our test data using these fitted models 
lg1.score(hdxtest, hdytest)
lg2.score(loytest, loytest)
#since these scores are rather good we can go ahead with training on our full data 

In [ ]:
lg3 = LogisticRegressionCV(Cs=10, cv=5, class_weight='balanced', max_iter=100, random_state=24)
lg4 = LogisticRegressionCV(Cs=10, cv=5, class_weight='balanced', max_iter=100, random_state=24)
lg3.fit(hd_x, hd_y)
lg4.fit(lo_x, lo_y)

In [ ]:
#adding predicted probabilities to our dataframes to show areas with highest predicted store presence in areas where 
# store count = 0 
hd_x2 = hd_x.assign(HD_dummy = hdlo.HD_dummy)
hd_x2 = hd_x2.assign(areaname = hdlo.areaname)
hd_x2 = hd_x2.loc[hd_x2['HD_dummy'] == 0]
hd_x2areaname = hd_x2.areaname
hd_x2.drop(['HD_dummy','areaname'], axis = 1, inplace = True)
hd_predictions = [x[1] for x in lg1.predict_proba_(hd_x2)]
hdanswers = list(zip(hd_x2areaname, list(hd_predictions)))
sorted(hdanswers, key = lambda x: x[1])[:5]


In [ ]:
#doing same for Lowes
lo_x2 = lo_x.assign(Lo_dummy = hdlo.Lo_dummy)
lo_x2 = lo_x2.assign(areaname = hdlo.areaname)
lo_x2 = lo_x2.loc[lo_x2['Lo_dummy'] == 0]
lo_x2areaname = lo_x2.areaname
hd_x2.drop(['Lo_dummy','areaname'], axis = 1, inplace = True)
lopredictions = [x[1] for x in lg4.predict_proba_(lo_x2)]
zippedanswer = list(zip(lo_x2areaname, list(lopredictions)))
sorted(zippedanswer, key = lambda x: x[1], reverse = True)

## Question 5. Where should “Tool Time” build its next 5 stores based on the Census Data on your customers? 

The logistic regression model predicts that Pinal, AZ; Ramsey, MN; Weld, CO; Webb, TX; and Ingham, MI are the top 5 locations to build new Tool Time stores.

**a. Explain your rational for your decision**

We want to find locations where both Home Depot and Lowes would consider building new stores, but we don't want to build in an area that is already saturated with other chains' stores. For this reason, I filtered the data by locations that had 1 or no stores for both chains. Then I sorted by probability that the location is a good place to build a store. This gives us locations that are promising for building stores, but have low stauration in terms of competition.

## Question 6. realtor.com market hotness index report 

**a. Using the realtor.com market hotness index report from August of 2018 create an additional variable to segment the country into the following regions**

In [ ]:
%ls

In [ ]:
# Keep only relevant columns from state_region
state_region = pd.read_csv('state_region.csv', sep = ',')
state_region.drop(['State', 'Division'], axis=1, inplace=True)

In [ ]:
#reading in our data
rdc = pd.read_csv('RDC_MarketHotness_Monthly.csv', sep = ',')
rdc['town'], rdc['state'] = rdc['ZipName'].str.split(',').str[0], rdc['ZipName'].str.split(', ').str[1]

In [ ]:
df1 = rdc.merge(state_region, how = 'left', left_on = 'state', right_on = 'State Code')
df1.drop('State Code', axis = 1, inplace = True)

In [ ]:
pd.set_option('display.max_columns', 36)

**b. Exploratory Data Analysis for realator.com data**

**i. Which Region of the country has the best “Demand Score”**

In [ ]:
df1.groupby('Region')['Demand Score'].mean()
#The Northeast has the best mean demand score 

**ii. Which State in the country has the best “Demand Score”**

In [ ]:
df1.groupby('state')['Demand Score'].aggregate({'mean'}).sort_values(by = 'mean', ascending = False)[:3]

Massachusetts has the best mean "Demand Score."

**iii. Which metro area (pop_2010 > 1million) has the best “Demand Score”**

In [ ]:
#merging our data from hdlo to extract population for each country
hdlo2 = hdlo[['county','pop_2010']]
df2 = df1.merge(hdlo2, how = 'inner', left_on = 'CountyFIPS', right_on = 'county')

In [ ]:
df2[df2.pop_2010 > 1e6][['CountyName', 'state','Demand Score']][:500]

Middlesex, MA has the best "Demand Score" of metro areas.

**c.	Compare and contrast these findings with your predicted new store findings.**

**i. Describe your findings as they relate to the customer attributes and potential business opportunity that Lowes, Home Depot and/or Tool Time may have if they are or are not located in the areas that have high demands for real estate opportunities**

It appears that the Northeast, Texas, Ohio, California, and Florida are all good areas for stores to be loacted, according to "Demand Score." This somewhat agrees with the model predictions; there were a lot of predicted locations in California and the Northeast.

**d. Add the following as features to the original HDLo data set and predict again where Tool Time should build its next 5 stores.**
1.	Median.Listing.Price
2.	Demand.Score
3.	Hotness.Score
4.	Nieleson.HH.Rank

In [ ]:
rdc2 = rdc[['CountyFIPS','Nielsen HH Rank','Demand Score','Hotness Score','Median Listing Price']]
hd_data = hdlo.merge(rdc2, how = 'inner', left_on = 'county', right_on = 'CountyFIPS')

In [ ]:
# Copy data into training and testing sets, drop earlier added probability columns along with other irrelevant columns
hd_data = hd_data.copy()
l_data = hd_data.copy()

hd_data.drop(['areaname', 'county', 'state', 'r1', 'r2', 'Lcount', 'HDcount', 'Lexists'], axis=1, inplace=True)
l_data.drop(['areaname', 'county', 'state', 'r1', 'r2', 'Lcount', 'HDcount', 'HDexists'], axis=1, inplace=True)

hd_X = hd_data.drop(['HD_dummy'], axis=1)
hd_y = hd_data.HD_dummy

l_X = l_data.drop(['Lo_dumm7'], axis=1)
l_y = l_data.Lo_dummy

# Split the data into train and test portions to test accuracy
hd_X_train, hd_X_test, hd_y_train, hd_y_test = train_test_split(hd_X, hd_y, random_state=42)
l_X_train, l_X_test, l_y_train, l_y_test = train_test_split(l_X, l_y, random_state=42)

In [ ]:
# Train the models
hd_logit = LogisticRegressionCV(Cs=10, cv=5, class_weight='balanced', max_iter=1000, random_state=42).fit(hd_X_train, hd_y_train)
l_logit = LogisticRegressionCV(Cs=10, cv=5, class_weight='balanced', max_iter=1000, random_state=42).fit(l_X_train, l_y_train)

In [ ]:
# Home Depot store predicition accuracy
hd_logit.score(hd_X_test, hd_y_test)

In [ ]:
# Lowes store predicition accuracy
l_logit.score(l_X_test, l_y_test)

In [ ]:
# Train on the full data
hd_full_logit = LogisticRegressionCV(Cs=10, cv=5, class_weight='balanced', max_iter=1500, random_state=42).fit(hd_X, hd_y)
l_full_logit = LogisticRegressionCV(Cs=10, cv=5, class_weight='balanced', max_iter=1500, random_state=42).fit(l_X, l_y)

In [ ]:
# Predict the target for all locations and extract the probability that a store should be built
hd_store_prob = [x[1] for x in hd_full_logit.predict_proba(hd_X)]
l_store_prob = [x[1] for x in l_full_logit.predict_proba(l_X)]

In [ ]:
# Add the probability features to the original dataset so we can find the best next locations to build stores
added_data['hd_store_prob'] = hd_store_prob
added_data['l_store_prob'] = l_store_prob

# Create new column that is the sum of the two probability columns for sorting purposes
added_data['prob_sum'] = added_data.hd_store_prob + added_data.l_store_prob

In [ ]:
# For Tool Time, show only locations where there are 1 or no stores for HD and Lowes,
# sort by prob_sum descending and show top 5 areas predicted to be the best store locations
added_data[(added_data.Lcount <= 1) & (added_data.HDcount <= 1)].sort_values(by='prob_sum', ascending=False).head(10)

**e. What are the top 5 new area names for which Tool Time should build their stores?**

The first four stores remain the same, however Ottawa, MI overtook Ingham, MI in this version of the model.

**f. Do these features increase the prediction accuracy for the new area predictions?**

Based on the new scores, the new features seem to have decreased the accuracy of the models.

**g. Does overlaying the realtor data set add value to the business strategy of Tool Time?**

In this case, I would conclude that adding the new features **does not** add value to the business strategy that I would recommend.

**h. Is there an alternative strategy that Tool Time should explore other than Census Data and Realtor data?**

Tool Time might want to consider areas that have high demand for commerical real estate, as new businesses will have contractors in need of supplies. Tool Time might also want to look into trends for up-and-coming neighborhoods where real estate prices may increase in the future, and get ahead of competitors by opening stores newly desired neighborhoods.